In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

# Load the dataset 
from google.colab import files

In [ ]:
uploaded= files.upload()
df = pd.read_csv('diabetes.csv')

Saving diabetes.csv to diabetes.csv


In [ ]:
print(df)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [ ]:
#no. of suffering and healthy
df.groupby('Outcome').size()

Outcome
0    500
1    268
dtype: int64

In [40]:
# Separate the features and labels
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=32)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Build the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units= 128, activation='relu'),
    tf.keras.layers.Dense(units=88, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # Monitor validation loss for learning rate reduction
    factor=0.3,            # Reduce learning rate by a factor of 0.1
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-7            # Minimum learning rate
)
# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with the learning rate reduction callback
model.fit(X_train, y_train,validation_data=(X_val,y_val),epochs=30, batch_size=22,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
best_model = tf.keras.models.load_model('best_model.h5')
loss, accuracy = best_model.evaluate(X_test, y_test)

print(loss, accuracy )

Epoch 1/30
18/18 [==============================] - 2s 20ms/step - loss: 0.5756 - accuracy: 0.7253 - val_loss: 46.7771 - val_accuracy: 0.3272 - lr: 0.0010
Epoch 2/30
18/18 [==============================] - 0s 5ms/step - loss: 0.4691 - accuracy: 0.7733 - val_loss: 84.5945 - val_accuracy: 0.3272 - lr: 0.0010
Epoch 3/30
18/18 [==============================] - 0s 6ms/step - loss: 0.4177 - accuracy: 0.7947 - val_loss: 72.2394 - val_accuracy: 0.3272 - lr: 0.0010
Epoch 4/30
18/18 [==============================] - 0s 6ms/step - loss: 0.3903 - accuracy: 0.8160 - val_loss: 94.8508 - val_accuracy: 0.3272 - lr: 0.0010
Epoch 5/30
18/18 [==============================] - 0s 6ms/step - loss: 0.3700 - accuracy: 0.8293 - val_loss: 93.9020 - val_accuracy: 0.3272 - lr: 0.0010
Epoch 6/30
8/8 [==============================] - 0s 3ms/step - loss: 0.5227 - accuracy: 0.7532
0.5227486491203308 0.7532467246055603
